# library import

In [29]:
import pandas as pd
import numpy as np
import os
import cv2

from sklearn.preprocessing import LabelEncoder

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.core.lightning import LightningModule
from pytorch_lightning import Trainer, seed_everything

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torchinfo import summary

from torchmetrics import functional as FM
from torchvision import transforms
from torch.utils.data import DataLoader, random_split, Dataset, TensorDataset

In [27]:
#https://wikidocs.net/73569

In [30]:
seed_everything(42, workers=True)

Global seed set to 42


42

# 정답 csv load

In [2]:
label_csv_path = 'Datasets/kid_f_train.csv'
label_data = pd.read_csv(label_csv_path) 

# class 확인 

In [3]:
count=label_data['name'].unique()
print("이름 : ", count)
print("총 개 수 : ", len(count))

이름 :  ['yujin' 'wonyoung' 'rei' 'choiyena' 'kimminju' 'joyuri' 'kanghyewon'
 'kwoneunbi' 'hondahitomi' 'sakura' 'kimchaewon' 'kazuha' 'huhyunjin'
 'hongeunchae' 'oliviahye' 'choilee' 'hyuna' 'kwonsohyun' 'hani' 'hyerin'
 'jeonghwa' 'le' 'solji' 'bora' 'dasom' 'hyolin' 'jeongyeon' 'jihyo'
 'mina' 'momo' 'nayeon' 'sana' 'karina' 'winter' 'giselle' 'ningning'
 'kimnamju' 'jisoo' 'jennie' 'rose' 'lisa' 'bravegirls_yuna'
 'bravegirls_yujeong' 'minyoung' 'bravegirls_eunji' 'hwasa' 'sola' 'joy'
 'yeri' 'sunny' 'sumin' 'seeun' 'yoon' 'chaewon' 'yeojin']
총 개 수 :  55


# 각 class마다 개수 파악

In [4]:
# 블랙핑크 몰빵 -> 데이터 샘플링 or class weight 써보기
label_data['name'].value_counts()

lisa                  1282
rose                  1245
jisoo                 1023
jennie                 414
kimminju               261
joyuri                 257
choiyena               135
hondahitomi            119
kimchaewon             119
kanghyewon              83
sakura                  79
nayeon                  76
oliviahye               72
kwoneunbi               70
yujin                   55
wonyoung                50
hongeunchae             35
ningning                28
karina                  25
giselle                 23
huhyunjin               22
winter                  21
kazuha                  18
bravegirls_yuna         16
minyoung                11
bravegirls_yujeong       6
hani                     6
bravegirls_eunji         6
jihyo                    3
momo                     2
hyerin                   2
chaewon                  2
seeun                    2
hyuna                    2
dasom                    1
hwasa                    1
sana                     1
y

# Label Encoder 

In [5]:
le = LabelEncoder()
name = le.fit_transform(label_data['name'])

In [6]:
print(name.shape)

(5591,)


In [7]:
print(name[0])

54


In [8]:
df = pd.DataFrame({'name' : name})
df['name'].value_counts()

33    1282
41    1245
21    1023
17     414
28     261
23     257
6      135
10     119
27     119
24      83
42      79
37      76
39      72
30      70
54      55
50      50
11      35
38      28
25      25
8       23
12      22
49      21
26      18
3       16
35      11
2        6
9        6
1        6
20       3
36       2
14       2
4        2
44       2
16       2
7        1
13       1
43       1
53       1
19       1
47       1
48       1
52       1
22       1
45       1
40       1
0        1
5        1
34       1
31       1
15       1
18       1
32       1
46       1
29       1
51       1
Name: name, dtype: int64

# image load

In [9]:
img_path = 'Datasets/HQ_512x512/HQ_512x512/'
dataset_size = len(label_data)
train_data = np.zeros((dataset_size,224,224,3), dtype=np.float32)


for i in range(0, dataset_size):
    img=cv2.imread(img_path+str(label_data['file_name'][i]))
    img_resize = cv2.resize(img, (224,224), interpolation=cv2.INTER_AREA)
    train_data[i] = img_resize / 255
    
print("총 이미지 개 수 : ", len(train_data))

총 이미지 개 수 :  5591


# 학습데이터 합치기

In [10]:
class CustomDataset(Dataset):
    def __init__(self, x_tensor, y_tensor):
        self.x = x_tensor
        self.y = y_tensor

    def __getitem__(self, index):
        return (self.x[index], self.y[index])

    def __len__(self):
        return len(self.x)

In [11]:
x_train_tensor = torch.from_numpy(train_data).float()  
y_train_tensor = torch.from_numpy(name).type(torch.LongTensor)  
x_train_tensor = x_train_tensor.permute(0,3,2,1).contiguous()
total_train_data = CustomDataset(x_train_tensor, y_train_tensor)

# VGG19 model 정의

In [33]:
class VGGNet_E(pl.LightningModule):
    def __init__(self, out_dim):
        super().__init__()
        
        #Convolutional layer
        self.cnn_layer_1 =nn.Sequential(
            nn.Conv2d(in_channels = 3,out_channels =64,kernel_size =3, stride =1, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels = 64,out_channels =64,kernel_size =3, stride =1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.cnn_layer_2 =nn.Sequential(
            nn.Conv2d(in_channels =64 ,out_channels =128 ,kernel_size =3 ,stride =1, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels =128 ,out_channels =128 ,kernel_size =3 ,stride =1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.cnn_layer_3 =nn.Sequential(
            nn.Conv2d(in_channels = 128, out_channels =256,kernel_size =3,stride = 1 , padding= 1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels = 256, out_channels =256,kernel_size =3,stride = 1 , padding= 1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels = 256, out_channels =256,kernel_size =3,stride = 1 , padding= 1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels = 256, out_channels =256,kernel_size =3,stride = 1 , padding= 1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.cnn_layer_4 =nn.Sequential(
            nn.Conv2d(in_channels = 256, out_channels =512,kernel_size =3,stride = 1 , padding= 1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels = 512, out_channels =512,kernel_size =3,stride = 1 , padding= 1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels = 512, out_channels =512,kernel_size =3,stride = 1 , padding= 1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels = 512, out_channels =512,kernel_size =3,stride = 1 , padding= 1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.cnn_layer_5 =nn.Sequential(
            nn.Conv2d(in_channels = 512, out_channels =512,kernel_size =3,stride = 1 , padding= 1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels = 512, out_channels =512,kernel_size =3,stride = 1 , padding= 1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels = 512, out_channels =512,kernel_size =3,stride = 1 , padding= 1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels = 512, out_channels =512,kernel_size =3,stride = 1 , padding= 1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        #FC Layer
        self.fc_layer_1=nn.Sequential(
            nn.Linear(512*7*7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5)
        )
        
        self.fc_layer_2=nn.Sequential(
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
        )
        
        self.fc_layer_3=nn.Sequential(
            nn.Linear(4096, out_dim)
            #Cross Entropy Loss에서 softmax를 포함한다.
            #nn.Softmax(dim=1)
        )
        
    def forward(self, x):
        #Convolutional layer
        x=self.cnn_layer_1(x)
        x=self.cnn_layer_2(x)
        x=self.cnn_layer_3(x)
        x=self.cnn_layer_4(x)
        x=self.cnn_layer_5(x)
        
        #FC layer
        x = x.view(-1, 512 * 7 * 7)
        x=self.fc_layer_1(x)
        x=self.fc_layer_2(x)
        x=self.fc_layer_3(x)
        
        return x
       
    def training_step(self, batch, batch_idx):
        x,y=batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        #y = y.to(torch.float32)
        y=y.long()
        pred_y = self(x)
        acc = FM.accuracy(pred_y, y)
        loss = F.cross_entropy(pred_y, y)
        metrics = {'val_acc': acc, 'val_loss': loss}
        self.log_dict(metrics , on_step=True, on_epoch=True, prog_bar=True, logger=True)
 
        return loss
    
    def test_step(self, batch, batch_idx):
        x, y = batch
        pred_y = self(x)
        acc = FM.accuracy(pred_y, y)
        loss = F.cross_entropy(pred_y, y)
        metrics = {'test_acc': acc, 'test_loss': loss}
        self.log_dict(metrics , on_step=True, on_epoch=True, prog_bar=True, logger=True)

        return loss
    
    def predict_step(self, batch, batch_idx):
        pass
    
    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(),
                              lr = 0.001)
                    
        lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)
        
        return [optimizer],[lr_scheduler]

# Datasets class 정의 

In [34]:
class Datasets(pl.LightningDataModule):
    def __init__(self, batch_size, datasets):
        super().__init__()
        self.batch_size = batch_size
        self.datasets = datasets
        self.transform = transforms.Compose([transforms.ToTensor()])
        
    def prepare_data(self):
        #데이터 load 하는 부분
        pass
    
    def setup(self, stage=None):
        
        if stage == 'fit':
            self.train_size = int(0.8 * len(self.datasets))
            self.test_size = len(self.datasets) - self.train_size
            self.train, self.val = random_split(self.datasets, [self.train_size, self.test_size])
        if stage == 'test':
            self.test = self.datasets
            
    def train_dataloader(self):
        return DataLoader(self.train, batch_size=self.batch_size)
    
    def val_dataloader(self):
        return DataLoader(self.val, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.test, batch_size=self.batch_size)

In [35]:
data = Datasets(batch_size = 16, datasets = total_train_data)

In [36]:
data.setup(stage='fit')

# checkpoint 설정

In [37]:
checkpoint_callback = ModelCheckpoint(
  dirpath="checkpoints",
  filename="best-checkpoint",
  save_top_k=1,
  verbose=True,
  monitor="val_loss",
  mode="min"
)

#early_stopping_callback = EarlyStopping(monitor='val_loss', patience=2)

# 학습 설정 

In [38]:
model =  VGGNet_E(out_dim = 55)

trainer = pl.Trainer(
                    accelerator='gpu',
                    devices=1,
                     max_epochs=10,
                     #callbacks=[checkpoint_callback, early_stopping_callback],
                     callbacks=[checkpoint_callback],
                     deterministic=True
                    )
trainer.fit(model, data)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type       | Params
-------------------------------------------
0 | cnn_layer_1 | Sequential | 38.7 K
1 | cnn_layer_2 | Sequential | 221 K 
2 | cnn_layer_3 | Sequential | 2.1 M 
3 | cnn_layer_4 | Sequential | 8.3 M 
4 | cnn_layer_5 | Sequential | 9.4 M 
5 | fc_layer_1  | Sequential | 102 M 
6 | fc_layer_2  | Sequential | 16.8 M
7 | fc_layer_3  | Sequential | 225 K 
-------------------------------------------
139 M     Trainable params
0         Non-trainable params
139 M     Total params
559.182   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Epoch 0, global step 280: 'val_loss' reached 2.34869 (best 2.34869), saving model to 'D:\\minsu\\DL\\VGGNet\\checkpoints\\best-checkpoint-v2.ckpt' as top 1


...

Epoch 7, global step 2240: 'val_loss' was not in top 1


Validation: 0it [00:00, ?it/s]

Epoch 8, global step 2520: 'val_loss' was not in top 1


Validation: 0it [00:00, ?it/s]

Epoch 9, global step 2800: 'val_loss' was not in top 1
`Trainer.fit` stopped: `max_epochs=10` reached.


In [ ]:
trainer.save_checkpoint("VGG_E.pth")